# Туториал по двухэтапной модели

In [1]:
import datetime
import scipy
import dill
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import pyarrow.parquet as pq
from zipfile import ZipFile
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import warnings
warnings.simplefilter('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 20)

from tqdm import tqdm
tqdm.pandas()  # <- added this line

In [2]:
pd.read_csv('../input/hack-the-cart/hist_data.csv')

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14
...,...,...,...,...,...,...,...,...
4529884,95619205,4620221347,2021-09-07 18:48:29,204351817,1.000,99.990,False,9.84
4529885,95619205,4620221347,2021-09-07 18:48:29,203523604,1.000,62.190,False,9.84
4529886,95619205,4620221347,2021-09-07 18:48:29,203566357,2.000,39.890,False,9.84
4529887,95619205,4620221347,2021-09-07 18:48:29,203566418,1.000,62.190,False,9.84


# Покупатели

In [3]:
users = pd.read_csv('../input/hack-the-cart/hist_data.csv')
#users.rename(columns={'id': 'user_id', 'cluster_id' : 'item_id'} , inplace=True)
users.head()

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14


In [4]:
users.dtypes

buyer_id               int64
pav_order_id           int64
created               object
item_id                int64
count                float64
price_sold           float64
flag_weight_goods       bool
weight               float64
dtype: object

In [5]:
users['created'] = pd.to_datetime(users['created'], infer_datetime_format=True)

In [6]:
users.head()

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14


In [7]:
users.dtypes

buyer_id                      int64
pav_order_id                  int64
created              datetime64[ns]
item_id                       int64
count                       float64
price_sold                  float64
flag_weight_goods              bool
weight                      float64
dtype: object

In [8]:
#Временные фичи

users['month'] = users['created'].dt.month
users['day'] = users['created'].dt.day
users['dayofweek'] = users['created'].dt.dayofweek
users['hour'] = users['created'].dt.hour

del users['created']

In [9]:
users

,buyer_id,pav_order_id,item_id,count,price_sold,flag_weight_goods,weight,month,day,dayofweek,hour
0,95203091,98506637863,202808329,1.000,79.990,False,11.14,7,1,3,0
1,95203091,98506637863,202953905,1.072,44.945,True,11.14,7,1,3,0
2,95203091,98506637863,203566452,1.000,69.990,False,11.14,7,1,3,0
3,95203091,98506637863,202820143,1.972,41.295,True,11.14,7,1,3,0
4,95203091,98506637863,204400422,1.000,269.990,False,11.14,7,1,3,0
...,...,...,...,...,...,...,...,...,...,...,...
4529884,95619205,4620221347,204351817,1.000,99.990,False,9.84,9,7,1,18
4529885,95619205,4620221347,203523604,1.000,62.190,False,9.84,9,7,1,18
4529886,95619205,4620221347,203566357,2.000,39.890,False,9.84,9,7,1,18
4529887,95619205,4620221347,203566418,1.000,62.190,False,9.84,9,7,1,18


In [10]:
users.head()

,buyer_id,pav_order_id,item_id,count,price_sold,flag_weight_goods,weight,month,day,dayofweek,hour
0,95203091,98506637863,202808329,1.000,79.990,False,11.14,7,1,3,0
1,95203091,98506637863,202953905,1.072,44.945,True,11.14,7,1,3,0
2,95203091,98506637863,203566452,1.000,69.990,False,11.14,7,1,3,0
3,95203091,98506637863,202820143,1.972,41.295,True,11.14,7,1,3,0
4,95203091,98506637863,204400422,1.000,269.990,False,11.14,7,1,3,0


In [11]:
users['total_price'] = users['count'] * users['price_sold']

In [12]:
users['is_day_flag'] = users['hour'].between(7,21)

In [13]:
users.head()

,buyer_id,pav_order_id,item_id,count,price_sold,flag_weight_goods,weight,month,day,dayofweek,hour,total_price,is_day_flag
0,95203091,98506637863,202808329,1.000,79.990,False,11.14,7,1,3,0,79.99000,False
1,95203091,98506637863,202953905,1.072,44.945,True,11.14,7,1,3,0,48.18104,False
2,95203091,98506637863,203566452,1.000,69.990,False,11.14,7,1,3,0,69.99000,False
3,95203091,98506637863,202820143,1.972,41.295,True,11.14,7,1,3,0,81.43374,False
4,95203091,98506637863,204400422,1.000,269.990,False,11.14,7,1,3,0,269.99000,False


In [14]:
#users.groupby('buyer_id', as_index=False).agg(avg_total_cost = ('total_price', 'mean'))
#users.groupby('buyer_id', as_index=False).agg(std_total_cost = ('total_price', 'std'))
#users.groupby('buyer_id', as_index=False).agg(the_most_expensive_purchase = ('total_price', 'max'))

#users.groupby('buyer_id', as_index=False).agg(favorite_day = ('dayofweek', lambda x: scipy.stats.mode(x)[0]))
#users.groupby('buyer_id', as_index=False).agg(is_day_most_flag = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]))

# usersg = users.groupby('buyer_id', as_index=False).agg(
#     avg_total_cost = ('total_price', 'mean'),
#     std_total_cost = ('total_price', 'std'),
#     the_most_expensive_purchase = ('total_price', 'max'),
#     favorite_day = ('dayofweek', lambda x: scipy.stats.mode(x)[0]),
#     is_day_most_flag = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]))

In [15]:
#usersg

In [16]:
def buyer_fs_generation(path):
    users = pd.read_csv(path)
    
    users['created'] = pd.to_datetime(users['created'], infer_datetime_format=True)
    
    users['month'] = users['created'].dt.month
    users['day'] = users['created'].dt.day
    users['dayofweek'] = users['created'].dt.dayofweek
    users['hour'] = users['created'].dt.hour
    
    del users['created']
    
    users['total_price'] = users['count'] * users['price_sold']
    users['is_day_flag'] = users['hour'].between(7,21)

    
    usersg = users.groupby('buyer_id', as_index=False).agg(
        avg_total_cost = ('total_price', 'mean'),
        std_total_cost = ('total_price', 'std'),
        the_most_expensive_purchase = ('total_price', 'max'),
        favorite_day = ('dayofweek', lambda x: scipy.stats.mode(x)[0]),
        favorite_item = ('item_id', lambda x: scipy.stats.mode(x)[0]),
        is_day_most_flag = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]))
    
    usersg.to_csv('user_features.csv', index=False)
    return usersg.head()

In [17]:
buyer_fs_generation('../input/hack-the-cart/hist_data.csv')

,buyer_id,avg_total_cost,std_total_cost,the_most_expensive_purchase,favorite_day,favorite_item,is_day_most_flag
0,94578442,152.759769,100.008995,393.990,6,202801709,True
1,94578475,142.550292,159.480331,848.990,5,202791192,True
2,94578533,104.827176,100.222633,599.980,4,202807066,True
3,94578595,216.666559,296.156788,1803.732,1,202799674,True
4,94578656,93.898496,70.958066,335.490,1,202838754,True


# Айтемы

In [18]:
items = pd.read_csv('../input/hack-the-cart/hist_data.csv')
#users.rename(columns={'id': 'user_id', 'cluster_id' : 'item_id'} , inplace=True)
items.head()

,buyer_id,pav_order_id,created,item_id,count,price_sold,flag_weight_goods,weight
0,95203091,98506637863,2021-07-01 00:03:44,202808329,1.000,79.990,False,11.14
1,95203091,98506637863,2021-07-01 00:03:44,202953905,1.072,44.945,True,11.14
2,95203091,98506637863,2021-07-01 00:03:44,203566452,1.000,69.990,False,11.14
3,95203091,98506637863,2021-07-01 00:03:44,202820143,1.972,41.295,True,11.14
4,95203091,98506637863,2021-07-01 00:03:44,204400422,1.000,269.990,False,11.14


In [19]:
items['created'] = pd.to_datetime(items['created'], infer_datetime_format=True)

items['month'] = items['created'].dt.month
items['day'] = items['created'].dt.day
items['dayofweek'] = items['created'].dt.dayofweek
items['hour'] = items['created'].dt.hour

items['is_day_flag'] = items['hour'].between(7,21)

del items['created']

items

,buyer_id,pav_order_id,item_id,count,price_sold,flag_weight_goods,weight,month,day,dayofweek,hour,is_day_flag
0,95203091,98506637863,202808329,1.000,79.990,False,11.14,7,1,3,0,False
1,95203091,98506637863,202953905,1.072,44.945,True,11.14,7,1,3,0,False
2,95203091,98506637863,203566452,1.000,69.990,False,11.14,7,1,3,0,False
3,95203091,98506637863,202820143,1.972,41.295,True,11.14,7,1,3,0,False
4,95203091,98506637863,204400422,1.000,269.990,False,11.14,7,1,3,0,False
...,...,...,...,...,...,...,...,...,...,...,...,...
4529884,95619205,4620221347,204351817,1.000,99.990,False,9.84,9,7,1,18,True
4529885,95619205,4620221347,203523604,1.000,62.190,False,9.84,9,7,1,18,True
4529886,95619205,4620221347,203566357,2.000,39.890,False,9.84,9,7,1,18,True
4529887,95619205,4620221347,203566418,1.000,62.190,False,9.84,9,7,1,18,True


In [20]:
unique_orders = items['pav_order_id'].nunique()

#items.groupby('item_id', as_index=False).agg(favorite_day = ('dayofweek', lambda x: scipy.stats.mode(x)[0]))
#items.groupby('item_id', as_index=False).agg(is_day_most_flag = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]))

#items.groupby('item_id', as_index=False).agg(avg_amout = ('count', 'mean'))
#items.groupby('item_id', as_index=False).agg(std_amout = ('count', 'std')) дает дофига NaN
#items.groupby('item_id', as_index=False).agg(max_amout = ('count', 'max'))

items.groupby('item_id', as_index=False).agg(percentage_in_orders = ('pav_order_id', lambda x: x.count()/unique_orders))

,item_id,percentage_in_orders
0,202782406,0.000359
1,202782407,0.000004
2,202782412,0.000013
3,202782414,0.000030
4,202782415,0.000078
...,...,...
52467,218175579,0.000004
52468,218199321,0.000004
52469,218217955,0.000004
52470,218217991,0.000004


In [21]:
def item_fs_generation(path):
    users = pd.read_csv(path)
    
    users['created'] = pd.to_datetime(users['created'], infer_datetime_format=True)
    
    users['month'] = users['created'].dt.month
    users['day'] = users['created'].dt.day
    users['dayofweek'] = users['created'].dt.dayofweek
    users['hour'] = users['created'].dt.hour
    
    del users['created']
    
    users['is_day_flag'] = users['hour'].between(7,21)

    unique_orders = items['pav_order_id'].nunique()
    
    usersg = users.groupby('buyer_id', as_index=False).agg(
        favorite_day = ('dayofweek', lambda x: scipy.stats.mode(x)[0]),
        is_day_most_flag = ('is_day_flag', lambda x: scipy.stats.mode(x)[0]),
        avg_amout = ('count', 'mean'),
        max_amout = ('count', 'max'),
        percentage_in_orders = ('pav_order_id', lambda x: x.count() / unique_orders)
    )
    
    usersg.to_csv('item_features.csv', index=False)
    return usersg.head()

In [22]:
item_fs_generation('../input/hack-the-cart/hist_data.csv')

,buyer_id,favorite_day,is_day_most_flag,avg_amout,max_amout,percentage_in_orders
0,94578442,6,True,1.556595,6.00,0.000160
1,94578475,5,True,1.288000,3.00,0.000164
2,94578533,4,True,2.295129,50.00,0.000268
3,94578595,1,True,1.612349,7.88,0.000186
4,94578656,1,True,1.200300,4.00,0.000346
